In [6]:
from deap import gp, creator, tools, base, algorithms
from random import randint, random, seed, getstate, setstate
from functools import partial
from itertools import repeat

pset = gp.PrimitiveSetTyped(name='MAIN',
                            in_types=repeat(int, 1),
                            ret_type=float)
pset.addPrimitive(name='add', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='sub', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='mul', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='div', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='pow', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='sqrt', primitive=None, ret_type=float, in_types=[float])
pset.addPrimitive(name='and', primitive=None, ret_type=bool, in_types=[bool, bool])
pset.addPrimitive(name='or', primitive=None, ret_type=bool, in_types=[bool, bool])
pset.addPrimitive(name='not', primitive=None, ret_type=bool, in_types=[bool])
pset.addPrimitive(name='tern', primitive=None, ret_type=float, in_types=[bool, float, float])
pset.addPrimitive(name='lt', primitive=None, ret_type=bool, in_types=[float, float])
pset.addPrimitive(name='gt', primitive=None, ret_type=bool, in_types=[float, float])
pset.addPrimitive(name='eq', primitive=None, ret_type=bool, in_types=[float, float])

pset.addEphemeralConstant(name='iconst', ret_type=float, ephemeral=partial(randint, 0, 10))
pset.addEphemeralConstant(name='rconst', ret_type=float, ephemeral=partial(random))
pset.addEphemeralConstant(name='bconst', ret_type=bool, ephemeral=partial(randint, 0, 1))

pset.addTerminal(name='original', terminal=None, ret_type=float)
pset.addTerminal(name='cost', terminal=None, ret_type=float)
pset.addTerminal(name='degree', terminal=None, ret_type=float)
pset.addTerminal(name='area', terminal=None, ret_type=float)

In [7]:
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMin)

/home/mpvreal/Code/Faculdade/tcc/deap/venv/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mpvreal/Code/Faculdade/tcc/deap/venv/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [8]:
import numpy as np
import subprocess as sp
import scipy.stats as st

CAMINHO = r'/home/mpvreal/Code/Faculdade/tcc/llvm/testes/x86-64'

def obter_scripts_benchmark(benchmark: str) -> dict[str, str]:
  """
  Obtém os scripts de compilação e execução do benchmark.
  """
  global CAMINHO
  
  return { 'compile': f'{CAMINHO}/{benchmark}/compile', 'run': f'{CAMINHO}/{benchmark}/runbench' }

def compilar_benchmark(heuristica: str, scripts: dict[str, str]) -> None:
  """
  Escreve função heurística da alocação em um arquivo e compila o benchmark.
  """
  with open('../HeuristicFunction.txt', 'w+') as f:
    f.write(heuristica)

  sp.run(scripts['compile'], shell=True)

def coletar_tempo_exec(scripts: dict[str, str]) -> float:
  """
  Executa o benchmark e coleta o tempo de execução.
  """
  tempo = sp.run(scripts['run'], 
                         shell=True, 
                         stdout=sp.PIPE).stdout.decode('utf-8').replace(',', '.').strip()

  return float(tempo)

def calcular_media_e_erro(dados, confianca=0.95):
  """
  Obtém a média e margem de erro de um conjunto de dados, com um intervalo de confiança de 95%.
  """
  a = 1.0 * np.array(dados)
  n = len(a)

  media, erro_medio = np.mean(a), st.sem(a)
  h = erro_medio * st.t.ppf((1 + confianca) / 2., n-1)

  return media, h

def avaliar_fitness(individual, benchmark: str):
  """
  Escreve a heurística em um arquivo, compila o benchmark e coleta o tempo de execução médio após
  30 rodadas.
  """
  heuristica = str(gp.PrimitiveTree(individual))
  scripts = obter_scripts_benchmark(benchmark)
  compilar_benchmark(heuristica, scripts)

  tempos = []
  for _ in range(30):
    tempos.append(coletar_tempo_exec(scripts))
  
  media, margem_erro = calcular_media_e_erro(tempos)

  return media,

In [9]:
from operator import attrgetter

def inicializar_individuo(classe, string, pset):
  """
  Inicializa um indivíduo com uma árvore de expressão a partir de uma string.
  """
  return classe(gp.PrimitiveTree.from_string(string, pset))

def inicializar_populacao(inicializar, populacao, pset):
  """
  Inicializa a população com árvores de expressão a partir do arquivo populacao.
  """
  with open(populacao, 'r') as f:
    pop = f.read().split('\n')
  gp.PrimitiveTree
  
  return [inicializar(i, pset) for i in pop]

tb = base.Toolbox()
tb.register('individual', inicializar_individuo, creator.Individual)
tb.register('population', inicializar_populacao, tb.individual, '../populacao.txt', pset)
tb.register('evaluate', avaliar_fitness, benchmark='519.lbm_r')
tb.register('select', tools.selTournament, tournsize=7)
tb.register('mate', gp.cxOnePoint)
tb.register('expr_mut', gp.genFull, min_=0, max_=2)
tb.register('mutate', gp.mutUniform, expr=tb.expr_mut, pset=pset)
tb.decorate('mate', gp.staticLimit(key=attrgetter('height'), max_value=17))
tb.decorate('mutate', gp.staticLimit(key=attrgetter('height'), max_value=17))

In [10]:
import pickle

CHECKPOINT_FREQ = 5 # Salvar a população a cada x gerações

def salvar_checkpoint(checkpoint, pop, gen, hof, log):
  global CHECKPOINT_FREQ
  if checkpoint and gen % CHECKPOINT_FREQ == 0:
    with open(checkpoint, 'wb') as arquivo_cp:
      cp = dict(population=pop, 
                generation=gen, 
                halloffame=hof,
                logbook=log,
                rndstate=getstate())
      pickle.dump(cp, arquivo_cp)

def evoluir(population, toolbox, mu, lambda_, cxpb, mutpb, ngen,
            stats=None, halloffame=None, verbose=__debug__, checkpoint=None, logbook=None):
  logbook_from_checkpoint = logbook is not None
  
  if not logbook_from_checkpoint:
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

  # Evaluate the individuals with an invalid fitness
  invalid_ind = [ind for ind in population if not ind.fitness.valid]
  fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
  for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit

  if halloffame is not None:
    halloffame.update(population)

  record = stats.compile(population) if stats is not None else {}
  if not logbook_from_checkpoint:  
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
      print(logbook.stream)

  # Begin the generational process
  for gen in range(1, ngen + 1):
    # Vary the population
    offspring = algorithms.varOr(population, toolbox, lambda_, cxpb, mutpb)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    # Update the hall of fame with the generated individuals
    if halloffame is not None:
      halloffame.update(offspring)

    # Select the next generation population
    population[:] = toolbox.select(population + offspring, mu)

    # Update the statistics with the new population
    record = stats.compile(population) if stats is not None else {}
    logbook.record(gen=gen, nevals=len(invalid_ind), **record)
    if verbose:
      print(logbook.stream)

    # Salvar a população a cada x gerações
    salvar_checkpoint(checkpoint, population, gen, halloffame, logbook)

  return population, logbook

seed(666)

if True:
  pop = tb.population()
  hof = tools.HallOfFame(1)
  start_gen = 0
  logbook = None

  stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
  stats_size = tools.Statistics(len)
  mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
  mstats.register('media', np.mean)
  mstats.register('dvp', np.std)
  mstats.register('min', np.min)
  mstats.register('max', np.max)
else:
  with open(checkpoint, "rb") as cp_file:
    cp = pickle.load(cp_file)
  pop = cp["population"]
  start_gen = cp["generation"]
  hof = cp["halloffame"]
  logbook = cp["logbook"]
  setstate(cp["rndstate"])

pop, log = evoluir(population=pop,
                   toolbox=tb,
                   cxpb=0.5,
                   mutpb=0.1,
                   mu=len(pop),
                   lambda_=(2 * len(pop)),
                   ngen=30 - start_gen,
                   stats=mstats,
                   halloffame=hof,
                   verbose=True,
                   logbook=logbook,
                   checkpoint='checkpoint.pkl')

function@LBM_allocateGrid,0,0,0,0,0,1,0,0,0,0,1
function@LBM_freeGrid,0,0,0,0,0,1,0,0,0,0,1
function@LBM_initializeGrid,0,0,0,0,0,0,0,0,0,0,0
function@LBM_swapGrids,0,0,0,0,0,0,0,0,0,0,0
function@LBM_loadObstacleFile,0,0,0,0,0,9,0,0,0,0,33860.5
function@LBM_initializeSpecialCellsForLDC,0,0,0,0,0,7,0,0,0,0,3740.12
function@LBM_initializeSpecialCellsForChannel,0,0,0,0,0,2,0,0,0,0,10954.5
function@LBM_performStreamCollideBGK,4,4,0,6,0,3,31.5,31.5,95.25,0,47.625
function@LBM_performStreamCollideTRT,28,27,0,49,0,11,444.5,420.625,777.875,0,158.625
function@LBM_handleInOutFlow,6,6,0,11,0,4,191.25,191.25,350.625,0,127.5
function@LBM_showGridStatistics,21,3,0,15,0,6,148.75,47.8125,252.938,0,19.9375
function@LBM_storeVelocityField,3,0,0,3,0,12,1088,0,1088,0,114694
function@LBM_compareVelocityField,22,1,0,23,0,14,623680,32768,656448,0,139270
function@main,0,0,0,0,0,14,0,0,0,0,54.1667
function@MAIN_parseCommandLine,0,0,0,0,0,3,0,0,0,0,3
function@MAIN_printInfo,0,0,0,0,0,0,0,0,0,0,0
function@MAIN_i

KeyboardInterrupt: 